In [1]:
import numpy as np
import pandas as pd

from env import user, host, password

import seaborn as sns
import matplotlib.pyplot as plt

import os

# Acquire
Acquire 
- `bedroomcnt`, 
- `bathroomcnt`, 
-`calculatedfinishedsquarefeet`, 
- `taxvaluedollarcnt`, 
- `yearbuilt`, 
- `taxamount`, and 
- `fips`

from the zillow database for all 'Single Family Residential' properties.

### Function to connect to Codeup mysql server

In [2]:
def get_connection(database_name):
    '''
    This function takes in a database name from the sequel server and returns a string with 
    the credentials needed to access that server.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{database_name}'

### Function to acquire data from server
1. created a variable to hold query
2. can be updated or modified if want to make changes later

In [3]:
# 1. query used to acquire data from sql

query = '''select bedroomcnt,
    bathroomcnt,
    calculatedfinishedsquarefeet,
    taxvaluedollarcnt,
    yearbuilt,
    taxamount,
    fips
    from properties_2017
    where propertylandusetypeid = 261;'''

In [4]:
# 2. function to acquire data (uses the query)

def get_zillow_data():
    '''
    This function reads the zillow database from the Codeup mysql server adn returns a dataframe
    If a local csv file does not exist, this function writes the data to a csv file
    The local file can be used as a backup in the event that you cannot access the mysql database
    '''
    
    
    if os.path.isfile('zillow.csv'):
# if a local file already exists
        df = pd.read_csv('zillow.csv', index_col = 0)
        
    else:
# if a local file does not yet exist
        df = pd.read_sql(query, get_connection('zillow'))

# and cache data into csv file
        df.to_csv('zillow.csv')
    
    return df

### Taking a look at the data

In [5]:
# peak at the data

zillow = get_zillow_data()
zillow.head()

/usr/local/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


# 2. Summarization and Cleaning

In [6]:
print(f'Rows: {zillow.shape[0]}\nColumns: {zillow.shape[1]}')

Rows: 2152863
Columns: 7


In [7]:
zillow.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


### Handling our null values

In [8]:
zillow.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 131.4 MB


In [9]:
# percentage of null values in each column to total values

print(f'{(zillow.isna().sum()) / (zillow.shape[0])}')


bedroomcnt                      0.000005
bathroomcnt                     0.000005
calculatedfinishedsquarefeet    0.003941
taxvaluedollarcnt               0.000229
yearbuilt                       0.004337
taxamount                       0.002063
fips                            0.000000
dtype: float64


### We have such a small percentage of nulls compared to total that we can drop those rows

In [10]:
# dropping null rows

zillow.dropna(inplace = True)

In [11]:
# confirming no null values in df

zillow.isna().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

### Cleaning up our column names and data types

In [12]:
# looking at the values for each column and creating a cols list
cols = []

for col in zillow.columns:
    cols.append(col)
    print(col)
    print(zillow[col].value_counts())
    print('------------------------')
    print()

bedroomcnt
3.0     962944
4.0     633608
2.0     334221
5.0     150671
6.0      25117
1.0      22895
7.0       4792
0.0       4397
8.0       1103
9.0        290
10.0       118
11.0        34
13.0        15
12.0        12
14.0         7
15.0         5
18.0         3
16.0         2
25.0         1
Name: bedroomcnt, dtype: int64
------------------------

bathroomcnt
2.00     942463
3.00     422398
1.00     412582
2.50     142827
4.00      82039
1.50      31157
3.50      28464
5.00      28306
4.50      19474
6.00      10717
5.50       6201
7.00       4381
0.00       4274
8.00       1681
6.50       1330
9.00        707
7.50        382
10.00       322
11.00       145
8.50        108
12.00        73
9.50         50
13.00        39
14.00        25
15.00        17
0.50         16
10.50        14
16.00        12
18.00         8
20.00         6
17.00         4
1.75          3
12.50         3
11.50         3
14.50         1
32.00         1
19.50         1
19.00         1
Name: bathroomcnt, dtype: i

In [13]:
cols

['bedroomcnt',
 'bathroomcnt',
 'calculatedfinishedsquarefeet',
 'taxvaluedollarcnt',
 'yearbuilt',
 'taxamount',
 'fips']

In [14]:
# renaming columns

zillow.rename(columns = {'bedroomcnt': 'bedrooms',
                         'bathroomcnt': 'bathrooms',
                         'calculatedfinishedsquarefeet': 'sqft',
                         'taxvaluedollarcnt': 'tax_value',
                         'yearbuilt': 'year_built',
                         'taxamount': 'tax_amt',
                         'fips': 'fips'}, inplace = True)

In [15]:
# preparing for loop that converts floats ------> to integers by creating 2 lists from columns
cols = zillow.columns
cols = cols.to_list()
print(f'All columns: {cols}\n')

flt_cols = ['bathrooms', 'sqft', 'tax_value', 'tax_amt']
int_cols = ['bedrooms', 'year_built', 'fips']
print(f'Float columns: {flt_cols}')
print(f'Integer columns: {int_cols}')

All columns: ['bedrooms', 'bathrooms', 'sqft', 'tax_value', 'year_built', 'tax_amt', 'fips']

Float columns: ['bathrooms', 'sqft', 'tax_value', 'tax_amt']
Integer columns: ['bedrooms', 'year_built', 'fips']


In [16]:
# for loop to convert floats to -----> int

for col in int_cols:
    zillow[col] = zillow[col].astype(int)

In [17]:
# confirming our dtypes have been updated

zillow.dtypes

bedrooms        int64
bathrooms     float64
sqft          float64
tax_value     float64
year_built      int64
tax_amt       float64
fips            int64
dtype: object

https://github.com/CodeupClassroom/germain-regression-exercises/blob/main/wrangle_exercise_review.ipynb
<br><i>referenced for loop from Germain regression wrange_exercises_review.ipynb solutions</i>

In [18]:
# # variable distributions

# plt.figure(figsize = (15, 5))

# for i, col in enumerate(cols):
# # i starts at 0, but I want the plot number to start at 1 ---> adding 1
#     plot_number = 1 + 1
  
    
#     plt.subplot(1, len(cols), plot_number)
    
# # title with column name
#     plt.title(col)
    
# # creating histogram
#     zillow[col].hist(bins = 5)
    



### Summary Takeaways:
>All but `fips` column has null values<br>
all variables are floats
can change following to int:
- `yearbuilt`
- `fips`